Given a class Example with Python bindings, it’s possible to return instances wrapped in C++11 unique pointers

In [ ]:
/** C++ */
std::unique_ptr<Example> create_example() { return std::unique_ptr<Example>(new Example()); }

In [ ]:
/* Pybind11 */
m.def("create_example", &create_example);

 However, it is illegal to use them as function arguments. 

In [ ]:
void do_something_with_example(std::unique_ptr<Example> ex) { ... }

The signature would imply that Python needs to give up ownership of an object that is passed to this function, which is generally not possible (for instance, the object might be referenced elsewhere).

It is possible to switch to other types of reference counting wrappers or smart pointers, which is useful in codebases that rely on them. For instance, the following snippet causes std::shared_ptr to be used instead.

In [ ]:
py::class_<Example, std::shared_ptr<Example> /* <- holder type */> obj(m, "Example");

Note that any particular class can only be associated with a single holder type.
One potential stumbling block when using holder types is that they need to be applied consistently. Can you guess what’s broken about the following binding code?

In [ ]:
class Child { };

class Parent {
public:
   Parent() : child(std::make_shared<Child>()) { }
   Child *get_child() { return child.get(); }  /* Hint: ** DON'T DO THIS ** */
private:
    std::shared_ptr<Child> child;
};

PYBIND11_MODULE(example, m) {
    py::class_<Child, std::shared_ptr<Child>>(m, "Child");

    py::class_<Parent, std::shared_ptr<Parent>>(m, "Parent")
       .def(py::init<>())
       .def("get_child", &Parent::get_child);
}

The problem is that Parent::get_child() returns a pointer to an instance of Child, but the fact that this instance is already managed by std::shared_ptr<...> is lost when passing raw pointers. 
In this case, pybind11 will create a second independent std::shared_ptr<...> that also claims ownership of the pointer. 
In the end, the object will be freed twice since these shared pointers have no way of knowing about each other.

There are two ways to resolve this issue:

    1, For types that are managed by a smart pointer class, never use raw pointers in function arguments or return values. In other words: always consistently wrap pointers into their designated holder types (such as std::shared_ptr<...>). In this case, the signature of get_child() should be modified as follows:

In [ ]:
std::shared_ptr<Child> get_child() { return child; }

    2, Adjust the definition of Child by specifying std::enable_shared_from_this<T> (see cppreference for details) as a base class. This adds a small bit of information to Child that allows pybind11 to realize that there is already an existing std::shared_ptr<...> and communicate with it. In this case, the declaration of Child should look as follows:

In [ ]:
class Child : public std::enable_shared_from_this<Child> { };

Custom smart pointers

pybind11 supports std::unique_ptr and std::shared_ptr right out of the box. 
For any other custom smart pointer, transparent conversions can be enabled using a macro invocation similar to the following. 
It must be declared at the top namespace level before any binding code

In [ ]:
/* Always needed for custom holder types */
PYBIND11_DECLARE_HOLDER_TYPE(T, SmartPtr<T>);

/* Only needed if the type's `.get()` goes by another name */
namespace pybind11 { namespace detail {
    template <typename T>
    struct holder_helper<SmartPtr<T>> { // <-- specialization
        static const T *get(const SmartPtr<T> &p) { return p.getPointer(); }
    };
}}